# Create NetCDF4 output - Explore BoXPol/JuXPol
<div class="alert alert-info">

**User Info** <br>
 Global Data - User Input 
 
 start_time, end_time, location and sweep
 
 location = "BoXPol" or "JuXPol"  (X-band radar)
 
 elev_id  = 0 ... 9   (IDs for elevation angles)
    
 For BoXPol, the elevation folders name change in 2017 (check for error message)
   
</div>
 

In [51]:
import datetime as dt
import glob
import os
import re

In [52]:
import numpy as np
import dask
import wradlib as wrl
import xarray as xr

In [53]:
def get_xpol_path(path=None, start_time=dt.datetime.today(), loc='boxpol'):
    """Create path of BoXPol/JuXPol radar data files
    
    Parameter
    ---------
    path : str
        Path to root folder of radar data, Defaults to None (/auotomount)
    
    Return
    ------
    radar_path : str
        Path to radar data
        
    """
    loc = "" if loc.lower()[0:2] == "bo" else "_juelich"
    if path is None:
        ins = "-archiv" if start_time < dt.datetime(2015,1,1) else "" 
        path = f"/automount/radar{ins}/scans{loc}"
    date = "{0}-{1:02d}-{2:02d}".format(start_time.year, start_time.month, start_time.day)
    radar_path = os.path.join(path, "{0}/{0}-{1:02}/{0}-{1:02d}-{2:02d}")
    return radar_path.format(start_time.year, start_time.month, start_time.day)

def get_file_date_regex(filename):
    """Get regex from filename
    """
    # regex for ""%Y-%m-%d--%H:%M:%S"
    reg0 = r"\d{4}.\d{2}.\d{2}..\d{2}.\d{2}.\d{2}"
    # regex for "%Y%m%d%H%M%S" 
    reg1 = r"\d{14}"
    match = re.search(reg0, os.path.basename(filename))
    return reg1 if match is None else reg0

def get_datetime_from_filename(filename, regex):
    """Get datetime from filename
    """
    fmt = "%Y%m%d%H%M%S"
    match = re.search(regex, os.path.basename(filename))
    match = "".join(re.findall(r"[0-9]+", match.group()))
    return dt.datetime.strptime(match, fmt)

def create_filelist(path_glob, starttime, endtime):
    """Create filelist from path_glob and filename dates
    """
    file_names = sorted(glob.glob(path_glob))
    regex = get_file_date_regex(file_names[0])
    for fname in file_names:
        time = get_datetime_from_filename(fname, regex)
        if time >= starttime and time < endtime:
            yield fname


# Global Data - USER INPUT

In [58]:
start_time = dt.datetime(2015, 7, 5, 0, 0)
end_time = dt.datetime(2015, 7, 5, 23, 59, 59)
location = "BoXPol"
elev_id  = 0
# User Input End #

if location == "BoXPol":
    snames = ["n_ppi_010deg","ppi_1p5deg","ppi_2p4deg","ppi_3p4deg","n_ppi_045deg","n_ppi_082deg", 
          "n_ppi_110deg","n_ppi_140deg","n_ppi_180deg","n_ppi_280deg"]
    # 2017 onwards, elevations changed ?
    #snames = ["n_ppi_010deg","n_ppi_020deg","n_ppi_031deg","n_ppi_045deg","n_ppi_060deg","n_ppi_082deg", 
    #      "n_ppi_110deg","n_ppi_140deg", "n_ppi_180deg", "n_ppi_280deg"]
    #
    sname  = snames[elev_id]
    sweep  = 0
elif location == "JuXPol":
    sname = "DWD_Vol_2"
    sweep = elev_id

output_path = ""

radar_path = get_xpol_path(start_time=start_time, loc=location)
radar_path

'/automount/radar/scans/2015/2015-07/2015-07-05'

# Create Netcdf Dataset of wanted radar data

In [59]:
def to_netcdf(ds, fpath, engine='netcdf4'):
    """Export Dataset to NetCDF file
    """
    delayed_obj = ds.to_netcdf(fpath, engine=engine, compute=False)
    from dask.diagnostics import ProgressBar
    with ProgressBar():
        results = delayed_obj.compute()

def save_netcdf_dataset(ds, path, post=None, engine='netcdf4'):
    """Save Dataset to NetCDF file 
    """
    elevation = np.median(ds.elevation)
    t0 = ds.time[0].values.astype("M8[s]").astype("O")
    t0 = t0.replace(second=0, microsecond=0, minute=((t0.minute // 5) * 5))
    t1 = ds.time[-1].values.astype("M8[s]").astype("O") + dt.timedelta(minutes=5)
    t1 = t1.replace(second=0, microsecond=0, minute=((t1.minute // 5) * 5))
    fname = [f"{ds.location.values}", f"{int(round(elevation, 1)*10)}", f"{t0:%Y%m%d}", f"{t0:%H%M}",
             f"{t1:%H%M}"]
    if post is not None:
        fname.append(post)
    fname = "_".join(fname) + ".nc"
    ofname = os.path.join(path, fname)
    ds.pipe(to_netcdf, ofname, engine=engine)
    return ofname

def create_volume(location, name, start_time, end_time, sweep, outpath=""):
    """Create NetCDF file from radar data
    """
    radar_path = get_xpol_path(start_time=start_time, loc=location)
    file_path = os.path.join(radar_path, name)
    file_obj = list(create_filelist(os.path.join(file_path, "*"), start_time, end_time))
    vol = wrl.io.open_odim(file_obj, loader="h5py", flavour="GAMIC", chunks={})
    return vol

def create_netcdf_dataset(location, name, start_time, end_time, sweep, outpath="", chunks={}, engine='h5netcdf'):
    """Create NetCDF file from radar data
    """
    radar_path = get_xpol_path(start_time=start_time, loc=location)
    file_path = os.path.join(radar_path, name)
    file_obj = list(create_filelist(os.path.join(file_path, "*"), start_time, end_time))
    vol = wrl.io.open_odim(file_obj, loader="h5py", flavour="GAMIC", chunks=chunks)
    ds = vol[sweep].data
    ds = ds.assign_coords({"location": location})
    #ds = ds.chunk({"time": 24})
    ofname = save_netcdf_dataset(ds, outpath, engine=engine)
    del vol
    return os.path.abspath(ofname)

# Create netcdf Dataset

In [56]:
%%time
fname_raw = create_netcdf_dataset(location, sname, start_time, end_time, sweep, outpath=output_path, engine='h5netcdf', chunks=None)
fname_raw

Collecting: 100%|██████████| 24/24 [00:01<00:00, 14.86 Timesteps/s]


[########################################] | 100% Completed |  0.1s
CPU times: user 7.1 s, sys: 136 ms, total: 7.24 s
Wall time: 7.98 s


'/data/hubhome/pshrestha/miubrt/notebooks/getting_started/BoXPol_180_20150705_0000_0200.nc'

# Load and Explore Dataset

In [37]:
import matplotlib as mpl
import hvplot
import hvplot.xarray
import holoviews as hv

## Load

In [38]:
swp = xr.open_dataset(fname_raw, chunks={"time": 12})

## Georeference

In [39]:
swp = swp.pipe(wrl.georef.georeference_dataset)

## Interactive Exploration

In [40]:
swp = swp.chunk()

In [41]:
dbz_plot = swp.hvplot.quadmesh(groupby='time',
                               x='x', y='y', 
                               z='DBZH', 
                               rasterize=True, 
                               clim=(-20,50), cmap=mpl.cm.nipy_spectral,
                               width=500, #height=300,
                               aspect=1,
                              )
dbz_plot = dbz_plot.options(color_levels=list(np.arange(-20, 51, 2)))
dbz_plot

:DynamicMap   [time]
   :Image   [x,y]   (DBZH)